In [1]:
# pip install llama-index llama-index-utils-workflow llama-index-llms-google-genai llama-index-tools-google

In [3]:
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="gemini-2.5-pro")

In [5]:
from google.genai import types

google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

llm_with_search = GoogleGenAI(
    model="gemini-2.5-pro",
    generation_config=types.GenerateContentConfig(tools=[google_search_tool])
)

In [6]:
response = llm_with_search.complete("What's the weather like today in Valencia, Spain?")
print(response)

## Sunny Skies and Warm Temperatures in Valencia Today

**Valencia, Spain is currently experiencing a sunny day with warm temperatures.** As of 12:34 PM local time, the temperature is approximately 29°C (84°F). The forecast for the rest of the day indicates continued sunshine with a maximum temperature around 31°C (89°F) and a minimum nighttime temperature of about 24°C (76°F).

There is a very low chance of rain throughout the day. Humidity levels are currently around 58%, and are expected to be around 69% during the day.

Looking ahead, the weather in Valencia is expected to remain sunny and warm for the next several days, with temperatures gradually increasing.


In [ ]:
from llama_index.core.workflow import Context

async def planning(ctx: Context, query: str) -> str:
    
    prompt = f"""Search in Internet and Create a detailed plan for execute a complete investigation about the specify topic from the query: "{query},"
    Please provide the research plan."""

    response = await llm_with_search.acomplete(prompt)
    
    current_state = await ctx.store.get("state")
    
    current_state["research_plan"] = {}
    current_state["research_plan"][query] = response.text
    await ctx.store.set("state", current_state)
    
    return response.text

async def search_web(ctx: Context, query: str) -> str:
    """Useful for searching the web about a specific query or topic"""
    response = await llm_with_search.acomplete(f"""Please research given this query or topic,
    and return the result\n<query_or_topic>{query}</query_or_topic>""")
    return response

async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic."""
    current_state = await ctx.store.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.store.set("state", current_state)
    return "Notes recorded."

async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic."""
    current_state = await ctx.store.get("state")
    current_state["report_content"] = report_content
    await ctx.store.set("state", current_state)
    return "Report written."

async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback."""
    current_state = await ctx.store.get("state")
    current_state["review"] = review
    await ctx.store.set("state", current_state)
    return "Report reviewed."

async def save_report(ctx: Context, review):
    current_state = await ctx.store.get("state")
    
    with open("report.md", "w", encoding="utf-8") as archivo:
            archivo.write(current_state["report_content"])
    
    return f"Contenido guardado exitosamente en report.md"

In [20]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

planning_agent = FunctionAgent(
    name="PlanningAgent",
    description="Useful for planning",
    system_prompt=(
        "You are the PlanningAgent that can make the plan for information on a given topic and send this plan to the research_agent. "
        "Once the planning are complete and you are satisfied, you should hand off control to the ResearchAgent to start searching info following your plan about the topic."
    ),
    llm=llm,
    tools=[planning, record_notes],
    can_handoff_to=["ResearchAgent"],
)

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review a report and provide feedback. "
        "Your feedback should either approve the current report or request changes for the WriteAgent to implement."
        "Once the review is done, you should save the report."
    ),
    llm=llm,
    tools=[review_report,save_report],
    can_handoff_to=["ResearchAgent","WriteAgent"],
)

In [21]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[planning_agent, research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "plan: Not written yet."
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

research_topic = """Write a report on the current landscape of AI and LLMs, specifically Gemini."""

handler = agent_workflow.run(
    user_msg=research_topic
)

current_agent = None
current_tool_calls = ""

async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: ResearchAgent

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'current landscape of AI and LLMs'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'current landscape of AI and LLMs'}
  Output: 
🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'AI and LLM landscape 2024'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'AI and LLM landscape 2024'}
  Output: ## AI and LLM Landscape in 2024: A Year of Accelerated Integration and Maturing Capabilities

The year 2024 has marked a pivotal moment in the evolution of artificial intelligence (AI) and large language models (LLMs), characterized by a distinct shift from experimental enthusiasm to widespread, production-grade implementation across industries. While groundbreaking innovations continue to emerge, the overarching theme of the year has been the practical application and scaling of these technologies, alongside a growing awa

In [11]:
state = await handler.ctx.store.get("state")
print("Report Content:\n", state["report_content"])
print("\n------------\nFinal Review:\n", state["review"])

Report Content:
 # The Shifting Sands of Intelligence: A Look at the Current AI and LLM Landscape, Featuring Google's Gemini

## Introduction

The world of Artificial Intelligence (AI) and Large Language Models (LLMs) is in a constant state of flux, characterized by rapid technological breakthroughs, substantial financial investment, and an evolving regulatory environment. The race for more powerful, efficient, and versatile models is dominated by a handful of tech giants, each pushing the boundaries of what was once thought possible. This report delves into the current landscape, with a special focus on Google's powerful Gemini model.

## The Key Players

The field is currently led by a few major players:

*   **OpenAI:** Continues to be a dominant force with its influential GPT series.
*   **Google:** Has made significant strides with its natively multimodal Gemini family and its open-source Gemma models.
*   **Meta:** Contributes significantly to the open-source community with its p